<a href="https://colab.research.google.com/github/1ucky40nc3/TREX/blob/testing/testing_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Aug 19 21:30:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# @title Utils for the entire Notebook
# @markdown ✋ Rerun Cell if Runtime was restarted 🔄

from IPython.utils.io import capture_output


def execute(func, *args, verbose: bool = False, **kwargs):
    if verbose:
        return func(*args, **{"verbose": verbose, **kwargs})
    
    with capture_output() as captured:
        return func(*args, **{"verbose": verbose, **kwargs})

---


# ***Natural Language Processing (NLP)*** 📰🤯


---

In [3]:
# @title | NLP | Install Dependencies ⇩
VERBOSE = False # @param {type:"boolean"}
    

def install_nlp_dependencies(**kwargs):
    !pip install sentencepiece
    !pip install transformers
    !pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html
    !pip install torch-geometric
    !pip install torch-scatter==2.0.8 -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html


execute(install_nlp_dependencies, verbose=VERBOSE)

In [29]:
# @title | NLP | Initialize the NLP Pipelines
# @markdown ✋ Rerun Cell if Runtime was restarted 🔄

import torch
import transformers

def TABLE_QA(*args, **kwargs):
    return {"answer": "WOW"}

def device(boolean: bool) -> int:
    return 0 if boolean else -1

#@markdown ---
TABLE_QA_MODEL = "google/tapas-base-finetuned-wikisql-supervised" #@param ["lysandre/tiny-tapas-random-wtq", "lysandre/tiny-tapas-random-sqa", "google/tapas-base-finetuned-wtq", "google/tapas-base-finetuned-sqa", "google/tapas-base-finetuned-wikisql-supervised", "google/tapas-large-finetuned-wtq", "google/tapas-large-finetuned-sqa", "google/tapas-large-finetuned-wikisql-supervised"]
USE_GPU_4_TABLE_QA = False # @param {type:"boolean"}

#@markdown ---
SMALL_TALK_MODEL = "facebook/blenderbot-90M" #@param ["facebook/blenderbot-90M", "facebook/blenderbot-400M-distill", "facebook/blenderbot-1B-distill", "facebook/blenderbot-3B"]
USE_GPU_4_SMALL_TALK = True # @param {type:"boolean"}

#@markdown ---
FEW_SHOT_MODEL = "EleutherAI/gpt-neo-125M" #@param ["gpt2", "gpt2-medium", "gpt2-large", "gpt2-xl", "EleutherAI/gpt-neo-125M", "EleutherAI/gpt-neo-1.3B", "EleutherAI/gpt-neo-2.7B"]
USE_GPU_4_FEW_SHOT = True # @param {type:"boolean"}

#@markdown ---
DEACTIVATE_TABLE_QA = False # @param {type:"boolean"}


#@markdown ---
LANGUAGE = "en" #@param ["en", "de"]

#@markdown ---
GERMAN_TO_ENGLISH_MODEL = "Helsinki-NLP/opus-mt-de-en" #@param ["Helsinki-NLP/opus-mt-de-en"]
USE_GPU_4_GERMAN_TO_ENGLISH = False # @param {type:"boolean"}

#@markdown ---
ENGLISH_TO_GERMAN_MODEL = "Helsinki-NLP/opus-mt-en-de" #@param ["Helsinki-NLP/opus-mt-en-de"]
USE_GPU_4_ENGLISH_TO_GERMAN = False # @param {type:"boolean"}

#@markdown ---
VERBOSE = False # @param {type:"boolean"}

def initialize_nlp_pipelines(**kwargs):
    print("[DEBUG] Downloading Zero-Shot-Classification Components")
    ZERO_SHOT = transformers.pipeline(
        "zero-shot-classification")

    print("[DEBUG] Downloading Small-Talk Components")
    SMALL_TALK = transformers.pipeline(
        "conversational", 
        model=SMALL_TALK_MODEL, 
        device=device(USE_GPU_4_SMALL_TALK))

    print("[DEBUG] Downloading Text-To-Text Components")
    FEW_SHOT = transformers.pipeline(
        "text-generation", 
        model=FEW_SHOT_MODEL, 
        device=device(USE_GPU_4_FEW_SHOT))


    if not DEACTIVATE_TABLE_QA:
        print("[DEBUG] Downloading Table QA Components")
        TABLE_QA = transformers.pipeline(
            "table-question-answering", 
            model=TABLE_QA_MODEL, 
            device=device(USE_GPU_4_TABLE_QA))
        
    if LANGUAGE == "de":
        print("[DEBUG] Downloading German-To-English Translation Components")
        GERMAN_TO_ENGLISH_TRANSLATOR = transformers.pipeline(
            "translation_de_to_en", 
            model=GERMAN_TO_ENGLISH_MODEL)
        print("[DEBUG] Downloading English-To-German Translation Components")
        ENGLISH_TO_GERMAN_TRANSLATOR = transformers.pipeline(
            "translation_en_to_de", 
            model=ENGLISH_TO_GERMAN_MODEL)
    
    return locals()

PIPELINES = execute(initialize_nlp_pipelines, verbose=VERBOSE)

In [8]:
# @title | NLP | Set up Services for Data
# @markdown ✋ Rerun Cell if Runtime was restarted 🔄

import io
import pandas as pd
import transformers


TRAIN_TABLE = """Location,Train,Start,Destination,Departure,Arrival,Delay
Munich,ICE 77,Munich,Berlin,12:30,13:33,2
Munich,ICE 56,Munich,Leipzig,12:30,13:30,4
Munich,ICE 33,Leipzig,Berlin,13:30,14:00,45"""

PLANE_TABLE = """Location,Plane,Start,Destination,Departure,Arrival,Delay
Munich,Eurowings (EW8003),Munich,Berlin,12:30,13:33,0
Munich,Lufthansa (LH1940),Munich,Leipzig,12:30,13:30,10
Munich,Lufthansa (LH2040),Leipzig,Berlin,13:30,14:00,7"""


def train_table() -> pd.DataFrame:
    return pd.read_csv(io.StringIO(TRAIN_TABLE))

def plane_table() -> pd.DataFrame:
    return pd.read_csv(io.StringIO(PLANE_TABLE))

def df_to_csv(df) -> str:
    csv = io.StringIO()
    df.to_csv(csv, index=False)
    return csv.getvalue()

def csv_to_df(csv) -> pd.DataFrame:
    return pd.read_csv(io.StringIO(csv))

def travel_tables(tables=[train_table(), plane_table()]) -> pd.DataFrame:
    travel = pd.concat(tables)
    travel = travel.fillna("NONE")

    csv = df_to_csv(travel)
    travel = csv_to_df(csv)
    travel = travel.astype(
        {column: str for column in travel.columns.values})
    return travel

travel_samples = """Question: "I am in Hannover. It is 17:48. Which train can I take from Munich to Berlin?" Context: "ICE 33" Answer: "The train ICE 33 will travel from Munich to Berlin."
Question: "I am in Frankfurt. It is 09:32. When will the next flight to Madrid leave?" Context: "07:59" Answer: "The next to Madrid will leave 07:59."
Question: "I am in Hannover. It is 17:48. What is the best train to get to Berlin from London?" Context: "ICE 33" Answer: "The train ICE 33 is the fastest."
Question: "I am in Munich. It is 05:51. From which platform does the RB 61 exit to Zurich?" Context: "8" Answer: "The RB 61 will departs from the platform 8."
Question: "I am in Berlin. It is 12:20. How can I get to London by aircraft?" Context: "Eurowings (EW8003)" Answer: "The flight Eurowings (EW8003) will leave for London."
Question: "I am in London. It is 12:20. I am at a train station. How can I get to Frankfurt?" Context: "ICE 923" Answer: "The train ICE 77 is available for your trip."
Question: "I am in Hamburg. It is 23:43. From which gate does the Lufthansa aircraft (LH1940) take off?" Context: "23" Answer: "The Lufthansa airliner (LH1940) will take off from gate 23."
Question: "I am in Stuttgart. It is 08:24. When will the next ICE to Bremen leave?" Context: "07:59" Answer: "The next to Bremen will leave 07:59."
Question: "I am in Bremen. It is 20:15. Is the ICE 1556 delayed?" Context: "9" Answer: "The ICE 1556 will be delayed by 9 minutes."
Question: "I am in Dresden. It is 10:17. How much is the flight LH1239 delayed?" Context: "59" Answer: "The flight LH1239 will be delayed by 59 minutes."
"""

def samples_length(samples: str, model: str) -> int:
    if not "EleutherAI/gpt-neo" in model:
        raise Exception(
            """The tokenizer of the FEW-SHOT model is not accessible!
            Therefore the sample length can not be computed.""")
        
    tokenizer = transformers.GPT2Tokenizer.from_pretrained(FEW_SHOT_MODEL)
    input_ids = tokenizer(travel_samples, return_tensors="pt").input_ids
    return input_ids.shape[-1]


TRAVEL_TIME = "7:00" #@param {type: "string"}
TRAVEL_LOCATION = "Munich" #@param {type: "string"}

def travel_time() -> str:
    return TRAVEL_TIME

def travel_location() -> str:
    return TRAVEL_LOCATION

travel_tables()

,Location,Train,Start,Destination,Departure,Arrival,Delay,Plane
0,Munich,ICE 77,Munich,Berlin,12:30,13:33,2,NONE
1,Munich,ICE 56,Munich,Leipzig,12:30,13:30,4,NONE
2,Munich,ICE 33,Leipzig,Berlin,13:30,14:00,45,NONE
3,Munich,NONE,Munich,Berlin,12:30,13:33,0,Eurowings (EW8003)
4,Munich,NONE,Munich,Leipzig,12:30,13:30,10,Lufthansa (LH1940)
5,Munich,NONE,Leipzig,Berlin,13:30,14:00,7,Lufthansa (LH2040)


In [9]:
# @title | NLP | NLP Implementation
# @markdown ✋ Rerun Cell if Runtime was restarted 🔄

#@markdown ---
VERBOSE = False # @param {type:"boolean"}

from typing import Dict
from typing import List
from typing import Tuple
from typing import Callable
from typing import Optional


class Associations(Dict):
    data: Callable[[], pd.DataFrame]
    samples: str

Response = Tuple[str, Optional[transformers.Conversation]]

Function = Callable[
    [transformers.Conversation], 
    Response
]

class Skill(Dict):
    associations: Associations
    function: Function

class Skills(Dict):
    name: str
    skill: Skill

WarmUp = Callable[
    [transformers.Conversation], 
    transformers.Conversation
]

"""~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Section for Classification on a Zero-Shot basis.
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"""

def zero_shot_classification(input: str, 
                             labels: List[str], 
                             top_k: Optional[int] = 1,
                             verbose: Optional[bool] = False,
                             **kwargs) -> List[str]:
    if verbose:
        print(f"[DEBUG] |Zero-Shot Classification| input: {input}")
        print(f"[DEBUG] |Zero-Shot Classification| labels: {labels}")
        print(f"[DEBUG] |Zero-Shot Classification| top_k: {top_k}")
    
    top_k_labels = PIPELINES["ZERO_SHOT"](
        input, labels)["labels"][:top_k]
    
    if verbose:
        print(f"[DEBUG] |Zero-Shot Classification| top_k_labels: {top_k_labels}")
    return top_k_labels

def skill_classification(input: str, 
                         skills: List[str], 
                         verbose: Optional[bool] = False,
                         **kwargs) -> List[str]:
    if verbose:
        print(f"[DEBUG] |Skill Classification| input: {input}")
        print(f"[DEBUG] |Skill Classification| skills: {skills}")

    skill = zero_shot_classification(input, skills, **kwargs)[0]

    if verbose:
        print(f"[DEBUG] |Skill Classification| skill: {skill}")
    return skill

def toxicity_classification(input: str,
                            verbose: Optional[bool] = False,
                            labels: List[str] = ["toxic", "non-toxic"],
                            **kwargs) -> List[str]:
    if verbose:
        print(f"[DEBUG] |Toxicity Classification| input: {input}")
        print(f"[DEBUG] |Toxicity Classification| labels: {labels}")

    label = zero_shot_classification(input, labels, **kwargs)[0]

    if verbose:
        print(f"[DEBUG] |Toxicity Classification| label: {label}")
    return label


"""~~~~~~~~~~~~~~~~~~~
Section for Table QA.
~~~~~~~~~~~~~~~~~~~"""

def table_question_answering(input: str, 
                             table: pd.DataFrame, 
                             verbose: Optional[bool] = False, 
                             **kwargs) -> str:
    if verbose:
        print(f"[DEBUG] |Table Question Answering| input: {input}")
        print(f"[DEBUG] |Table Question Answering| table: \n{table}")
    
    output = PIPELINES["TABLE_QA"](table=table, query=input)

    if verbose:
        print(f"[DEBUG] |Table Question Answering| output: \n{output}")
    return output["answer"]


"""~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Section for Few-Shot Text Generation
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"""

def few_shot(query: str, 
             samples: str, 
             verbose: Optional[bool] = False, 
             **kwargs) -> str:
    if verbose:
        print(f"[DEBUG] |Few-Shot Text Generation| query: \n{query}")
        print(f"[DEBUG] |Few-Shot Text Generation| samples: \n{samples}")
    
    output = PIPELINES["FEW_SHOT"](samples + query, **kwargs)

    if verbose:
        print(f"[DEBUG] |Few-Shot Text Generation| output: \n{output}")
    return output[0]["generated_text"]


def travel_few_shot(query: str, 
                    samples: str, 
                    verbose: Optional[bool] = False, 
                    **kwargs) -> str:
    if verbose:
        print(f"[DEBUG] |Travel Few-Shot Text Generation| query: \n{query}")
        print(f"[DEBUG] |Travel Few-Shot Text Generation| samples: \n{samples}")

    output = few_shot(query, 
                      samples, 
                      verbose, 
                      **kwargs)
    output = output[len(samples + query):]
    output = output.split('"')[0]

    if verbose:
        print(f"[DEBUG] |Travel Few-Shot Text Generation| output: \n{output}")
    return output


"""~~~~~~~~~~~~~~~~~~~~~~~~~
Section for Skill Functions
~~~~~~~~~~~~~~~~~~~~~~~~~"""

def travel_skill(conversation: transformers.Conversation, 
                 associations: Associations, 
                 verbose: Optional[bool] = False, 
                 **kwargs) -> Response:
    if verbose:
        print(f"[DEBUG] |Travel Skill| input conversation: \n{conversation}")
        print(f"[DEBUG] |Travel Skill| associations: \n{associations}")

    input = conversation.new_user_input
    labels = list(associations.keys())

    variant = zero_shot_classification(input, labels, **kwargs)[0]

    data = associations[variant]["data"]()
    time = associations[variant]["time"]()
    location = associations[variant]["location"]()
    samples = associations[variant]["samples"]
    config = associations[variant]["config"]

    cell = table_question_answering(input, data, verbose)

    input = f"I am in {location}. It is {time}. {input}"
    query = f'Question: "{input}" Context: "{cell}" Answer: "'

    output = travel_few_shot(
        query, 
        samples, 
        verbose, 
        **{**config, **kwargs})

    if verbose:
        print(f"[DEBUG] |Travel Skill| variant: {variant}")
        print(f"[DEBUG] |Travel Skill| cell: {cell}")
        print(f"[DEBUG] |Travel Skill| output: {output}")
    return output, None

def small_talk_skill(conversation: transformers.Conversation, 
                     verbose: Optional[bool] = False, 
                     **kwargs) -> Response:
    if verbose:
        print(f"[DEBUG] |Small Talk Skill| input conversation: \n{conversation}")

    conversation = PIPELINES["SMALL_TALK"](conversation)
    output = conversation.generated_responses[-1]

    if verbose:
        print(f"[DEBUG] |Small Talk Skill| output conversation: \n{conversation}")
        print(f"[DEBUG] |Small Talk Skill| output: {output}")
    return output, conversation


"""~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Section for Personas and Warm Up
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~"""


def warm_up(conversation: transformers.Conversation, 
            personas: List[str],
            verbose: Optional[bool] = False,
            **kwargs) -> transformers.Conversation:
    for persona in personas:
        conversation.add_user_input(persona)
        conversation = PIPELINES["SMALL_TALK"](conversation)

    if verbose:
        print(f"[DEBUG] |Warm Up| personas: {personas}")
        print(f"[DEBUG] |Warm Up| personas: {conversation}")
    return conversation


"""~~~~~~~~~~~~~~~~~
Language Processors
~~~~~~~~~~~~~~~~~"""

def german_to_english_translation(input: str,
                                  verbose: Optional[bool] = False, 
                                  **kwargs) -> str:
    if verbose:
        print(f"[DEBUG] |German-To-English Translation| input: {input}")

    translation = PIPELINES["GERMAN_TO_ENGLISH_TRANSLATOR"](input, **{"num_beams": 40, **kwargs})

    if verbose:
        print(f"[DEBUG] |German-To-English Translation| translation: {translation}")
    return translation[0]["translation_text"]

def english_to_german_translation(input: str,
                                  verbose: Optional[bool] = False, 
                                  **kwargs) -> str:
    if verbose:
        print(f"[DEBUG] |English-To-German Translation| input: {input}")

    translation = PIPELINES["ENGLISH_TO_GERMAN_TRANSLATOR"](input, **{"num_beams": 40, **kwargs})

    if verbose:
        print(f"[DEBUG] |English-To-German Translation| translation: {translation}")
    return translation[0]["translation_text"]


"""~~~~~~~~~~~~~~~
NLP Configuration
~~~~~~~~~~~~~~~"""

LANGUAGES = {
    "en": {
        "preprocessor": lambda x: x,
        "postprocessor": lambda x: x,
    },
    "de": {
        "preprocessor": german_to_english_translation,
        "postprocessor": english_to_german_translation,
    }
}

TRAVEL_ASSOCIATIONS = {
    "travel": {
        "data": travel_tables,
        "time": travel_time,
        "location": travel_location,
        "samples": travel_samples,
        "config": {
            "temperature": 0.6,
            "do_sample": True,
            "max_length": samples_length(
                travel_samples, 
                FEW_SHOT_MODEL) + 50,
        }
    }
}

TRAVEL_SKILL = {
    "associations": TRAVEL_ASSOCIATIONS, 
    "function": travel_skill
}

SKILLS: Skills = {
    "small talk": {
        "associations": {}, 
        "function": small_talk_skill
    },
    "travel": TRAVEL_SKILL,
    "travel on time": TRAVEL_SKILL,
    "travel delayed": TRAVEL_SKILL,
    "other": {
        "associations": {}, 
        "function": small_talk_skill
    },
}

PERSONAS = []


CONFIG = {
    "languages": LANGUAGES,
    "personas": PERSONAS,
    "skills": SKILLS,
} 


class NLP:
    def __init__(self,
                 config: dict = CONFIG,
                 verbose: bool = True,
                 **kwargs):
        self.config = config

        language_processors = config["languages"][LANGUAGE]
        self.language_preprocessor = language_processors["preprocessor"]
        self.language_postprocessor = language_processors["postprocessor"]

        self.conversation = transformers.Conversation()
        self.conversation = warm_up(
            self.conversation, 
            personas=config["personas"],
            verbose=verbose,
            **kwargs)

        self.skills = config["skills"]
        
    def __call__(self, 
                 input: str, 
                 verbose: bool = False, 
                 **kwargs) -> str:
        if verbose:
            print(f"[DEBUG] |NLP __call__ START|" + "~"*20)
            print(f"[DEBUG] |NLP ATTR skills|: {self.skills}")
            print(f"[DEBUG] |NLP ATTR conversation|: \n{self.conversation}")
            print(f"[DEBUG] |NLP User input|: {input}")
        
        input = self.language_preprocessor(input)
        self.conversation.add_user_input(input)

        skill = skill_classification(
            input, 
            list(self.skills.keys()), 
            verbose=verbose, 
            **kwargs)
        
        pipeline = self.skills[skill]
        function = pipeline["function"]
        associations = pipeline["associations"]
        
        output, conversation = function(
            self.conversation, 
            associations=associations, 
            verbose=verbose,
            **kwargs)

        if conversation:
            self.conversation = conversation
        output = self.language_postprocessor(output)

        if verbose:
            print(f"[DEBUG] |NLP conversation |: \n{self.conversation}")
            print(f"[DEBUG] |NLP User output|: {output}")
            print(f"[DEBUG] |NLP __call__ END|" + "~"*20)
        return output

nlp = execute(NLP, verbose=VERBOSE, config=CONFIG)

---
---


# ***T-REX*** 🦖


---
---

In [13]:
# @title | T-REX | Start new Conversation
# @markdown ✋ Rerun Cell if Runtime was restarted 🔄

#@markdown ---
ACTIVATE_PERSONAS = False # @param {type:"boolean"}
PERSONA_1 = "I work in a travel agency" # @param {type:"string"}
PERSONA_1 = f"your persona: {PERSONA_1}"
PERSONA_2 = "My name is Mia" # @param {type:"string"}
PERSONA_2 = f"your persona: {PERSONA_2}"

#@markdown ---
VERBOSE = False # @param {type:"boolean"}


def trex_setup(**kwargs):
    config = {
        "languages": LANGUAGES,
        "personas": [PERSONA_1, PERSONA_2] if ACTIVATE_PERSONAS else [],
        "skills": SKILLS,
    }

    nlp = NLP(config=config, **kwargs)
    return nlp

nlp = execute(trex_setup, verbose=VERBOSE)

# Test the Toolkit

In [14]:
# @title Utils for Testing
# @markdown ✋ Rerun Cell if Runtime was restarted 🔄


from typing import Any
from typing import Tuple
from typing import List


def test_toolkit_component(
    component: Any, 
    dataset: List[Tuple[Any]], 
    config: dict = {}) -> dict:
    results = {**locals()}

    predictions = []
    for x, y in dataset:
        predictions.append(
            component(x, **config))
    
    results["predictions"] = predictions
    return results

def get_nlp_models(
    order: List[str] = [
        "SMALL_TALK", 
        "FEW_SHOT", 
        "TABLE_QA", 
        "SOURCE_TO_NATIVE_TRANSLATOR", 
        "NATIVE_TO_SOURCE_TRANSLATOR"],
    delimiter: str = ",") -> str:
    models = {
        "SMALL_TALK": SMALL_TALK_MODEL,
        "FEW_SHOT": FEW_SHOT_MODEL,
        "TABLE_QA": TABLE_QA_MODEL,
        "SOURCE_TO_NATIVE_TRANSLATOR": GERMAN_TO_ENGLISH_MODEL,
        "NATIVE_TO_SOURCE_TRANSLATOR": ENGLISH_TO_GERMAN_MODEL,
    }
    return delimiter.join([models[i] for i in order])

In [ ]:
# @title Test the **Zero-Shot Classification** Component

test_toolkit_component(
    zero_shot_classification, 
    [
        ("Hello, how are you?", "small-talk")
    ], 
    {
        "labels": ["travel", "small-talk"]
    })

In [ ]:
# @title Test the **Skill Classification** capability of the *Zero-Shot Classification* Component

test_toolkit_component(
    skill_classification,
    [
        ("Hello, how are you?", "small talk"),
        ("When will the ICE 77 arrive?",
            ["travel", "travel on time", "travel delayed"]),
    ],
    {
        "skills": list(SKILLS.keys()),
        "verbose": False
    }
)

In [ ]:
# @title Test the **Toxicity Classification** capability of the *Zero-Shot Classification* Component

test_toolkit_component(
    toxicity_classification,
    [
        ("Hello, how are you?", "non-toxic"),
        ("You bastard!", "toxic"),
    ],
    {
        "verbose": False,
    }
)

In [ ]:
# @title Test the **Table-QA** Component

test_toolkit_component(
    table_question_answering,
    [
        ("When will the ICE 77 departure?", "12:30"),
        ("How much will the ICE 77 be delayed?", "2"),
    ],
    {
        "table": travel_tables()
    }
)

In [ ]:
# @title Test the **Few-Shot** Component

test_toolkit_component(
    few_shot,
    [
        ("Hello, how are you?", None),
    ],
    {
        "samples": "",
    }
)

In [ ]:
# @title Test the **Travel Few-Shot** capability of the *Few-Shot* Component

test_toolkit_component(
    travel_few_shot,
    [
        ('Question: "I am in Munich. It is 7:00. When will the ICE 77 departure?" Context: "12:30" Answer: "', 'The ICE 77 will departure 12:30."')
    ],
    {
        "samples": travel_samples,
        **TRAVEL_ASSOCIATIONS["travel"]["config"]
    }
)

In [ ]:
# @title Test the **Travel-Skill**

test_toolkit_component(
    travel_skill,
    [
        (transformers.Conversation("When will the ICE 77 departure?"),
            "The ICE 77 will departure 12:30."),
    ],
    {
        "associations": TRAVEL_ASSOCIATIONS
    }
)

In [ ]:
# @title Test the **Small-Talk-Skill**

test_toolkit_component(
    small_talk_skill,
    [
        (transformers.Conversation("Hello, how are you?"), None),
    ],
    {}
)

In [ ]:
# @title Test the **Personas** of the *Small-Talk-Skill*

test_toolkit_component(
    warm_up,
    [
        (transformers.Conversation(), None),
    ],
    {
        "personas": [PERSONA_1, PERSONA_2],
    }
)

In [ ]:
# @title Test the **Soure-To-Native Translation** Component

test_toolkit_component(
    german_to_english_translation,
    [
        ("Wie geht es dir?", "How are you?"),
    ],
    {}
)

In [ ]:
# @title Test the **Native-To-Source Translation** Component

test_toolkit_component(
    english_to_german_translation,
    [
        ("How are you?", "Wie geht es dir?"),
    ],
    {}
)